In [3]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 가상의 데이터 1000개 생성 (기상 및 산불 위험 변수)
np.random.seed(42)
n_samples = 1000

# 가상의 기상 및 산불 위험 변수 생성 (실제 학습 데이터의 분포와 유사하게)
TA_AVG = np.random.uniform(-10, 30, n_samples)         # 평균 기온(°C)
HM_AVG = np.random.uniform(10, 90, n_samples)          # 평균 습도(%)
WS_AVG = np.random.uniform(0.5, 15, n_samples)         # 평균 풍속(m/s)
effective_humidity = np.random.uniform(20, 80, n_samples)  # 실효습도(%)
wind_speed = np.random.uniform(1, 15, n_samples)       # 풍속(m/s) (위험예보 데이터 기준)

# 실제 산불 발생 여부(타겟)는 모델이 예측할 것이므로, 임의로 산불 발생(1) 비율을 고려해 생성
# 예시: 산불 발생(1) 10% 정도로 설정 (실제 평가와 유사하게)
fire_occurred = np.random.choice([0, 1], size=n_samples, p=[0.9, 0.1])

df_virtual = pd.DataFrame({
    'TA_AVG': TA_AVG,
    'HM_AVG': HM_AVG,
    'WS_AVG': WS_AVG,
    'effective_humidity': effective_humidity,
    'wind_speed': wind_speed,
    'fire_occurred': fire_occurred
})

# 모델 불러오기 (XGBoost 네이티브 방식)
model = XGBClassifier()
model.load_model('xgb_fire_model_smote.json')  # 모델 파일 경로 확인[1]

features = ['TA_AVG', 'HM_AVG', 'WS_AVG', 'effective_humidity', 'wind_speed']
X_virtual = df_virtual[features].astype(float)
y_virtual = df_virtual['fire_occurred']

# 예측
preds_virtual = model.predict(X_virtual)

# 모델 평가
print("정확도:", accuracy_score(y_virtual, preds_virtual))
print("\n분류 리포트:")
print(classification_report(y_virtual, preds_virtual))
print("\n혼동 행렬:")
print(confusion_matrix(y_virtual, preds_virtual))

# 예측 결과 요약
result_df = pd.DataFrame({
    'actual_fire_occurred': y_virtual,
    'predicted_fire_occurred': preds_virtual
})
print("\n예측 결과 요약:")
print(result_df['predicted_fire_occurred'].value_counts())


정확도: 0.886

분류 리포트:
              precision    recall  f1-score   support

           0       0.89      0.99      0.94       892
           1       0.12      0.01      0.02       108

    accuracy                           0.89      1000
   macro avg       0.51      0.50      0.48      1000
weighted avg       0.81      0.89      0.84      1000


혼동 행렬:
[[885   7]
 [107   1]]

예측 결과 요약:
predicted_fire_occurred
0    992
1      8
Name: count, dtype: int64
